<a href="https://colab.research.google.com/github/nssn96/ML_GestureRecognition/blob/main/ML_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 4 - Gesture Recognition using HMM(Hidden Markov models)


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [103]:
# #install hmmlearn using pip3
# !pip install hmmlearn

In [104]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import pickle
from hmmlearn import hmm

In [105]:
#References Used


In [106]:
#Importing the dataset
dataset = pd.read_csv('Original_Dataset.csv')
print(dataset)

     LC_X_1  LC_X_2  LC_X_3  LC_X_4  LC_X_5  LC_X_6  LC_X_7  LC_X_8  LC_X_9  \
0       307     389     259     259     259     259     259     258     258   
1       340     340     340     340     339     339     340     339     338   
2       260     263     266     266     270     271     271     271     271   
3       338     339     338     337     337     337     337     337     336   
4       319     240     240     318     241     318     318     318     318   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
345       0     309     309     309     309     309     309     309     309   
346     301     300     302     302     302     301     299     300     301   
347     298     295     294     292     292     290     290     288     287   
348     278     278     278     277     278     277     276     276     276   
349     279     281     281     280     280     280     280     280     281   

     LC_X_10  ...  RD_22  RD_23  RD_24  RD_25  RD_2

In [107]:
dataset.columns

Index(['LC_X_1', 'LC_X_2', 'LC_X_3', 'LC_X_4', 'LC_X_5', 'LC_X_6', 'LC_X_7',
       'LC_X_8', 'LC_X_9', 'LC_X_10',
       ...
       'RD_22', 'RD_23', 'RD_24', 'RD_25', 'RD_26', 'RD_27', 'RD_28', 'RD_29',
       'RD_30', 'Gesture'],
      dtype='object', length=781)

In [108]:
''' Since we may not need all of that information and will probably perform well
using only the hand centroids, we will be dropping some of the columns from the dataset. We will take only take the x and y values
of the hands. '''
#Normalize each sequence and dropping some column data
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

dropped_columns = []
for column in dataset.columns:
    if str(column).find('LF') != -1 or  str(column).find('RF') != -1 or\
            str(column).find('LD') != -1 or str(column).find('RD') != -1:
        dropped_columns.append(column)
    elif column != 'Gesture':
        dataset[column] = normalize(dataset[column])
dataset = dataset.drop(dropped_columns,axis=1)
gestures = set(dataset.Gesture)
#print(dataset)
print(gestures)

{'Press', 'MoveUp', 'ZoomOut', 'MoveLeft', 'MoveRight', 'ZoomIn', 'MoveDown'}


In [109]:
# splitting the datset into train and test set--> I am splitting 90% train and 10% test set
basic_template = [0,0,0,0,0,0,0]  # here the size is 7 because, we have seven different classes
x_train = basic_template.copy()
x_test = basic_template.copy()
y_train = basic_template.copy()
y_test = basic_template.copy()
groupby_gesture = dataset.groupby(dataset.Gesture)
print(type(groupby_gesture))
for i, gesture in enumerate(gestures):
  gesture_data = groupby_gesture.get_group(gesture)
  x_train[i],x_test[i],y_train[i],y_test[i] = train_test_split(gesture_data.iloc[: , :120],gesture_data['Gesture'],test_size=0.2, random_state=1,stratify = gesture_data['Gesture'])
print('shape of train data:', np.array(x_train).shape)
print('shape of test data:',np.array(x_test).shape )

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
shape of train data: (7, 40, 120)
shape of test data: (7, 10, 120)


In [110]:
print(dataset)

       LC_X_1    LC_X_2    LC_X_3    LC_X_4    LC_X_5    LC_X_6    LC_X_7  \
0    0.647679  0.818947  0.546414  0.545263  0.546414  0.545263  0.545263   
1    0.717300  0.715789  0.717300  0.715789  0.715190  0.713684  0.715789   
2    0.548523  0.553684  0.561181  0.560000  0.569620  0.570526  0.570526   
3    0.713080  0.713684  0.713080  0.709474  0.710970  0.709474  0.709474   
4    0.672996  0.505263  0.506329  0.669474  0.508439  0.669474  0.669474   
..        ...       ...       ...       ...       ...       ...       ...   
345  0.000000  0.650526  0.651899  0.650526  0.651899  0.650526  0.650526   
346  0.635021  0.631579  0.637131  0.635789  0.637131  0.633684  0.629474   
347  0.628692  0.621053  0.620253  0.614737  0.616034  0.610526  0.610526   
348  0.586498  0.585263  0.586498  0.583158  0.586498  0.583158  0.581053   
349  0.588608  0.591579  0.592827  0.589474  0.590717  0.589474  0.589474   

       LC_X_8    LC_X_9   LC_X_10  ...   RC_Y_22   RC_Y_23   RC_Y_24  \
0  